PREPROCESSING
1.categorical feature
2.contineous feature 
3.encoding
4.handleing outliers
5.feature scaling

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
file_path = r"C:\Users\Mirafra\Mirafra\ML PROJECT PRACTICE\Project-Budget-Analyzer\notebook\data\data_final.csv"

# Load CSV into DataFrame
df = pd.read_csv(file_path)
df.shape

(6760, 32)

In [4]:

replace_map = {
    "unified modelling language (uml)": "uml",
    "ca erwin (aet)": "er modelling",
    "erwin": "er modelling",
    "entity relationship modelling": "er modelling",
    "prototyping was of screens": "prototyping",
    "prototype includes screens & reports": "prototyping",
    "prototyping was screens & reports": "prototyping",
    "prototyping was screens only": "prototyping",
    "prototyped screens": "prototyping",
    "prototyped database structure": "prototyping",
    "sap r/3": "sap",
    "seer method": "seer",
    "gane/sarson structured analysis": "structured analysis"
    
}

def clean_dev_techniques(text):
    if pd.isna(text):
        return []
    # split, lowercase, strip spaces
    items = [i.strip().lower() for i in text.split(";") if i.strip() != ""]
    # apply replacement mapping
    items = [replace_map.get(i, i) for i in items]
    # remove duplicates
    items = list(set(items))
    return items

# Apply cleaning
df["Cleaned_Techniques"] = df["Development Techniques"].apply(clean_dev_techniques)

print(df["Cleaned_Techniques"])


0                           [prototyping, data modelling]
1                                                      []
2                                                      []
3                                                      []
4                                                      []
                              ...                        
6755                                                   []
6756                                [structured analysis]
6757    [object oriented, data modelling, prototyping,...
6758    [object oriented analysis, business area model...
6759                                                   []
Name: Cleaned_Techniques, Length: 6760, dtype: object


In [5]:
# Flatten all lists into one set
all_labels = set([tech for sublist in df["Cleaned_Techniques"] for tech in sublist])

# Convert to sorted list for readability
all_labels = sorted(all_labels)

print("Number of unique techniques:", len(all_labels))
print(all_labels)

Number of unique techniques: 96
['(eg performance) & verify architecture', 'acceptance testing', 'activity modelling', 'basic analysis & design techniques', 'business area modelling', 'client specific process', 'cmmi', 'code inspections', 'conceptual analysis', 'configuration management: endevor', 'configuration management: sourcesafe', 'contingency management: backups', 'cost/benefit analysis', 'data & application conversion', 'data modelling', 'datarun', 'development model', 'doa', 'effect correspondence diagrams', 'effort estimates', 'er modelling', 'event modelling', 'evolutionary with weekly demo', 'experience pro', 'expressone methodology (client customized process)', 'full life cycle testing', 'functional decomposition', 'functional sizing', 'functional spec developed in conjunction with users', 'gantt charts, rad reviews (as milestones)', 'in prototyping, screens developed with users', 'in-house, project life cycle', 'internal analysis & design standards as per method', 'interv

In [8]:
categories = {
    "Testing": ["Unit Testing", "Regression Testing", "Integration Testing", "Testing-Oriented"],
    "Prototyping": ["Prototyping", "Rapid Prototyping", "Simulation"],
    "Analysis/Design": ["Data Modelling", "Process Modelling", "OO Analysis", "System Analysis"],
    "Methodology": ["Waterfall", "Agile", "Spiral", "RAD", "XP"],
    "Management": ["Project Management", "Risk Analysis", "Cost Estimation", "Planning"],
    "Tools/Config": ["CASE Tools", "Configuration Management", "Version Control", "Documentation"]
}

# Create reverse lookup for quick mapping
category_map = {tech: cat for cat, techs in categories.items() for tech in techs}

# Function to clean and categorize
def categorize_techniques(tech_string):
    if pd.isna(tech_string):
        return None
    techniques = [t.strip() for t in tech_string.split(";") if t.strip()]
    mapped = [category_map.get(t, "Other") for t in techniques]
    return "; ".join(mapped)

# Apply mapping
df["Category"] = df["Development Techniques"].apply(categorize_techniques)

# Check first 20 rows
print(df[["Development Techniques", "Category"]].head(20))

                               Development Techniques  \
0                         Data Modelling;Prototyping;   
1                                                 NaN   
2                                                 NaN   
3                                                 NaN   
4                                                 NaN   
5                                                 NaN   
6                                                 NaN   
7                                                 NaN   
8                                                 NaN   
9                                                 NaN   
10                                                NaN   
11                                                NaN   
12                                                NaN   
13  Data Modelling;Regression Testing;Testing-Orie...   
14                                                NaN   
15                  Data Modelling;Process Modelling;   
16                             

In [9]:
# Work only on the "Development Methodologies" column
df = df.explode("Development Methodologies")

# Before explode we need to split the string values on ";"
df["Development Methodologies"] = df["Development Methodologies"].dropna().str.split(";")

# Explode to make sure one methodology per row
df = df.explode("Development Methodologies")

# Strip whitespace and remove empty strings
df["Development Methodologies"] = df["Development Methodologies"].str.strip()
df = df[df["Development Methodologies"] != ""]

# Reset index
df = df.reset_index(drop=True)

print(df["Development Methodologies"].unique())
print("Number of Categories:", df["Development Methodologies"].nunique())

[nan 'Waterfall (includes Linear Processing)' 'Multifunctional Teams'
 'Waterfall (incl Linear Processing & SSADM)'
 'Joint Application Development (JAD)'
 'Rapid Application Development (RAD)' 'Timeboxing' 'Incremental'
 'Agile Development' 'Spiral' 'Iterative' 'Unified Process'
 'Personal Software Process (PSP)' 'Scrum' 'Lean' 'OCE' 'Interactive'
 'Extreme Programming (XP)' 'IMES OOM' 'IT Unified Process (ITUP)']
Number of Categories: 19


In [10]:
import re

def clean_methodology(m):
    if pd.isna(m):
        return None
    # Remove bracket terms
    m = re.sub(r"\(.*?\)", "", m).strip()

    # Normalize related Agile terms
    agile_aliases = [
        "Scrum", "Extreme Programming", "XP", "Lean", 
        "Timeboxing", "Incremental", "Iterative", 
        "Unified Process", "IT Unified Process"
    ]
    if any(alias.lower() in m.lower() for alias in agile_aliases):
        return "Agile Development"

    # Waterfall
    if "Waterfall" in m:
        return "Waterfall"

    return m

# Apply cleaning
df["Development Methodologies"] = df["Development Methodologies"].apply(clean_methodology)

# Drop None
df = df.dropna(subset=["Development Methodologies"]).reset_index(drop=True)

# Unique values
print(df["Development Methodologies"].unique())
print("Number of Categories:", df["Development Methodologies"].nunique())


['Waterfall' 'Multifunctional Teams' 'Joint Application Development'
 'Rapid Application Development' 'Agile Development' 'Spiral'
 'Personal Software Process' 'OCE' 'Interactive' 'IMES OOM']
Number of Categories: 10


In [ ]:
import pandas as pd
import re

# Load dataset
df = pd.read_csv(r"C:\Users\Mirafra\Mirafra\ML PROJECT PRACTICE\Project-Budget-Analyzer\notebook\data\data_final.csv")

# Function to clean methodologies
def clean_methodology(value):
    if pd.isna(value):
        return None
    
    # Split by ; and clean each
    methods = [m.strip() for m in value.split(";") if m.strip()]
    
    cleaned = []
    for m in methods:
        # Remove bracket terms
        m = re.sub(r"\(.*?\)", "", m).strip()
        
        # Normalize mappings
        if m.lower() in ["scrum", "extreme programming (xp)", "lean"]:
            m = "Agile Development"
        elif "unified process" in m.lower() or "it unified process" in m.lower():
            m = "Unified Process"
        elif "waterfall" in m.lower():
            m = "Waterfall"
        elif "jad" in m.lower():
            m = "Joint Application Development"
        elif "rad" in m.lower():
            m = "Rapid Application Development"
        elif "incremental" in m.lower():
            m = "Incremental"
        elif "spiral" in m.lower():
            m = "Spiral"
        elif "iterative" in m.lower():
            m = "Iterative"
        elif "psp" in m.lower():
            m = "Personal Software Process"
        elif "timeboxing" in m.lower():
            m = "Timeboxing"
        elif "multifunctional teams" in m.lower():
            m = "Multifunctional Teams"
        elif "oce" in m.lower():
            m = "OCE"
        elif "interactive" in m.lower():
            m = "Interactive"
        elif "imes oom" in m.lower():
            m = "IMES OOM"
        
        cleaned.append(m)
    
    # Keep only unique single methodology
    if cleaned:
        return ";".join(sorted(set(cleaned)))
    return None

# Apply cleaning function
df["Cleaned_Methodology"] = df["Development Methodologies"].apply(clean_methodology)

# Save back
df.to_csv(r"C:\Users\Mirafra\Mirafra\ML PROJECT PRACTICE\Project-Budget-Analyzer\notebook\data\data_final.csv", index=False)

print(df["Cleaned_Methodology"].unique())


[None 'Waterfall' 'Multifunctional Teams'
 'Joint Application Development;Multifunctional Teams;Rapid Application Development;Timeboxing'
 'Incremental' 'Joint Application Development;Multifunctional Teams'
 'Rapid Application Development' 'Agile Development' 'Spiral'
 'Joint Application Development;Rapid Application Development' 'Iterative'
 'Joint Application Development;Multifunctional Teams;Rapid Application Development'
 'Joint Application Development' 'Timeboxing'
 'Multifunctional Teams;Rapid Application Development'
 'Multifunctional Teams;Rapid Application Development;Timeboxing'
 'Multifunctional Teams;Timeboxing'
 'Multifunctional Teams;Unified Process'
 'Joint Application Development;Multifunctional Teams;Timeboxing'
 'Unified Process' 'Agile Development;Unified Process'
 'Agile Development;OCE;Unified Process' 'Multifunctional Teams;Waterfall'
 'OCE' 'OCE;Unified Process' 'Agile Development;Iterative' 'Interactive'
 'Joint Application Development;Rapid Application Developm

In [ ]:
# Apply cleaning function
df["Cleaned_Methodology"] = df["Development Methodologies"].apply(clean_methodology)

# Remove everything after ';'
df["Cleaned_Methodology"] = df["Cleaned_Methodology"].str.split(";").str[0].str.strip()

# Save back (overwrite the same CSV)
df.to_csv(
    r"C:\Users\Mirafra\Mirafra\ML PROJECT PRACTICE\Project-Budget-Analyzer\notebook\data\data_final.csv",
    index=False
)

# Check unique cleaned values
print(df["Cleaned_Methodology"].unique())
# Anuroop

[None 'Waterfall' 'Multifunctional Teams' 'Joint Application Development'
 'Incremental' 'Rapid Application Development' 'Agile Development'
 'Spiral' 'Iterative' 'Timeboxing' 'Unified Process' 'OCE' 'Interactive'
 'Extreme Programming']
